In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # Choose Quantized llama-3-8b-instructed model by unsloth to save GRAM
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
# sample of dataprocessing
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a chatbot for people who want to kill time"},
    {"role": "human", "content": "Hello, nice to meet you"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs))

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a chatbot for people who want to kill time<|eot_id|><|start_header_id|>human<|end_header_id|>\n\nHello, nice to meet you<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHello there! I'm happy to help you kill some time. What's on your mind? Do you want to chat about something in particular, or just shoot the breeze? I'm all ears (or rather, all text).<|eot_id|>"]


In [2]:
# Add lora adapter to the model
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
# Load the training and evaluation dataset (the eval dataset is same as origin llama3-8b-instruct)
import pickle
with open('data/llama3trainingdataset.pkl', 'rb') as f:
    training_dataset = pickle.load(f)

In [4]:
training_dataset = [{'conversations': i} for i in training_dataset]

In [5]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

training_dataset = Dataset.from_list(training_dataset)

In [6]:
train_test_split = training_dataset.train_test_split(test_size=0.166666, shuffle=False) # dataset already shuffled

train_dataset = train_test_split['train'] # 50000 instance
valid_dataset = train_test_split['test']  # 10000 instance
print(train_dataset)
print(valid_dataset)

Dataset({
    features: ['conversations'],
    num_rows: 50000
})
Dataset({
    features: ['conversations'],
    num_rows: 10000
})


In [7]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3"
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
# inspect the format of three datasets, all should looks the same
print(train_dataset[0])
print('---------------')
print(valid_dataset[0])

{'conversations': [{'content': "You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with prefered game's name, no need for explanation", 'role': 'system'}, {'content': 'I played two games counterstrike: source, and fallout: a post nuclear role playing game. After playing, I gave reviews for both games as follow:\ncounterstrike: source: good gmod texture pack\nfallout: a post nuclear role playing game: this games mom is a critical failure', 'role': 'user'}, {'content': 'counterstrike: source', 'role': 'assistant'}], 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with prefered game's name, no need for explanation<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI played two games counterstrike: source, and fallout

In [9]:
import re

def normalize_title(title):
    title = title.replace('.', '')
    title = title.replace(',', '')
    title = title.replace('/', '')
    title = title.replace(' ', '')
    title = title.replace("'", '')
    title = title.replace("-", '')
    # Remove non-ascii characters
    title = title.encode("ascii", errors="ignore").decode()
    # Remove extra spaces
    title = re.sub(' +', ' ', title)
    # Convert to lowercase
    title = title.lower()
    # Strip leading and trailing spaces
    title = title.strip()
    return title

In [ ]:
#test
correct_counter = 0
for i in range(1000):
    msg = valid_dataset[i]['conversations'][:2]
    #print(msg)
    inputs = tokenizer.apply_chat_template(
        msg,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
    outputs = tokenizer.batch_decode(outputs)

    answer = outputs[0].split('\n\n')[-1][:-10]
    correct_answer = valid_dataset[i]['conversations'][2]['content']
    if normalize_title(answer) == normalize_title(correct_answer):
        correct_counter += 1
    else:
        print(answer + ' | ' + correct_answer)

print(correct_counter)

In [10]:
# define trainer with parameters
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 0,
        max_steps = 1,
        learning_rate = 1e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        output_dir = "outputs",
        save_total_limit=5,  # Save only the top 5 models
        load_best_model_at_end=True,  # Load the best model at the end of training
        metric_for_best_model="loss",  # Use loss to determine the best model
        greater_is_better=False,  # Lower loss is better
        evaluation_strategy="steps",  # Set evaluation strategy to steps
        save_strategy="steps",  # Set save strategy to steps
        eval_steps=1,  # Evaluate every 10 steps (set this as needed)
    ),
)

/home/yuchen/miniconda3/envs/unsloth/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
5.594 GB of memory reserved.


In [12]:
validation_losses = []
for i in range(300):
    trainer.train()
    history = trainer.state.log_history
    validation_loss = [entry['eval_loss'] for entry in history if 'eval_loss' in entry]
    validation_losses = validation_losses + validation_loss
    model.save_pretrained(f"ll3_fitting/epoch_{i}")  

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 1
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,3.591300,3.658901


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 1
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,3.585000,3.658101


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 1
 "-____-"     Number of trainable parameters = 41,943,040


...

Step,Training Loss,Validation Loss
1,1.875800,2.451802


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 1
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,1.870500,2.443872


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 1
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,1.855400,2.438692


In [13]:
validation_losses

[3.6589014530181885,
 3.6581010818481445,
 3.657193422317505,
 3.658031940460205,
 3.6577672958374023,
 3.6573824882507324,
 3.654958486557007,
 3.654810905456543,
 3.6538538932800293,
 3.655472755432129,
 3.6550421714782715,
 3.65348744392395,
 3.654498338699341,
 3.654832124710083,
 3.6522231101989746,
 3.650686264038086,
 3.652634382247925,
 3.6503026485443115,
 3.6505284309387207,
 3.6498844623565674,
 3.6487035751342773,
 3.646705389022827,
 3.6466128826141357,
 3.6465682983398438,
 3.645169258117676,
 3.645540952682495,
 3.641561269760132,
 3.643056631088257,
 3.640719413757324,
 3.6403937339782715,
 3.6382434368133545,
 3.637108564376831,
 3.63653826713562,
 3.6367640495300293,
 3.633563280105591,
 3.6327147483825684,
 3.631845235824585,
 3.6296608448028564,
 3.6312835216522217,
 3.629044532775879,
 3.6267378330230713,
 3.627833127975464,
 3.6266324520111084,
 3.6249897480010986,
 3.6231467723846436,
 3.6214821338653564,
 3.6202831268310547,
 3.6177008152008057,
 3.6157312393188